In [1]:
from keras.models import load_model
from metrics import Pearson
from utils import prepare_data
import tensorflow_probability as tfp
import tensorflow as tf
import numpy as np
from dataloaders import GeneDataLoader

In [2]:
train_data, valid_data, test_data = prepare_data()

In [3]:
model = load_model('/Users/jonasflor/Downloads/Baseline_struct.h5',
                   custom_objects={'Pearson': Pearson})

In [4]:
dataloader = GeneDataLoader(test_data, padding_length=train_data['seq'].apply(lambda x: len(x)).max(), shuffle=False, struct=True)

prediction = []
y_test = []

for i, j in dataloader:
    prediction.append(model.predict(np.delete(i[0], 5, axis=2)))
    y_test.append(j)

prediction = np.concatenate(prediction)
y_test = np.concatenate(y_test)

1/1 [==============================] - 0s 48ms/step


In [5]:
def pearson_dict(y_true, y_pred):
    y_true_std = tfp.stats.stddev(y_true, sample_axis=0, keepdims=True)
    y_pred_std = tfp.stats.stddev(y_pred, sample_axis=0, keepdims=True)
    
    y_true /= (y_true_std + 1e-3)
    y_pred /= (y_pred_std + 1e-3)
    
    result = tfp.stats.covariance(x=y_true,
                                y=y_pred,
                                event_axis=None,
                                sample_axis=0,
                                keepdims=False)
    res_dict = {}
    res_dict['ERM'] = result[0]
    res_dict['KDEL'] = result[1]
    res_dict['LMA'] = result[2]
    res_dict['MITO'] = result[3]
    res_dict['NES'] = result[4]
    res_dict['NIK'] = result[5]
    res_dict['NLS'] = result[6]
    res_dict['NUCP'] = result[7]
    res_dict['OMM'] = result[8]
    return res_dict

In [6]:
pearson_dict(y_test, prediction)

{'ERM': <tf.Tensor: shape=(), dtype=float32, numpy=0.2852756>,
 'KDEL': <tf.Tensor: shape=(), dtype=float32, numpy=0.3411369>,
 'LMA': <tf.Tensor: shape=(), dtype=float32, numpy=0.13344598>,
 'MITO': <tf.Tensor: shape=(), dtype=float32, numpy=0.045985475>,
 'NES': <tf.Tensor: shape=(), dtype=float32, numpy=0.11924936>,
 'NIK': <tf.Tensor: shape=(), dtype=float32, numpy=0.17616875>,
 'NLS': <tf.Tensor: shape=(), dtype=float32, numpy=0.15459798>,
 'NUCP': <tf.Tensor: shape=(), dtype=float32, numpy=0.1914392>,
 'OMM': <tf.Tensor: shape=(), dtype=float32, numpy=0.08259771>}

In [7]:
np.mean(list(pearson_dict(y_test, prediction).values()))

0.16998856